# ROM-SHRED Tutorial on Kuramoto Sivashinsky

#### Import Libraries

In [71]:
# PYSHRED
%load_ext autoreload
%autoreload 2
from pyshred import ParametricDataManager, SHRED, SHREDEngine

# Other helper libraries
import matplotlib.pyplot as plt
from scipy.io import loadmat
import torch
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Load Kuramoto Sivashinsky dataset

In [ ]:
import numpy as np
import urllib.request
# URL of the NPZ file
url = 'https://zenodo.org/records/14524524/files/KuramotoSivashinsky_data.npz?download=1'
# Local filename to save the downloaded file
filename = 'KuramotoSivashinsky_data.npz'
# Download the file from the URL
urllib.request.urlretrieve(url, filename)
# Load the data from the NPZ file
dataset = np.load(filename)

#### Initialize Data Manager

In [66]:
# Initialize ParametricSHREDDataManager
manager = ParametricDataManager(
    lags = 21,
    train_size = 0.8,
    val_size = 0.1,
    test_size = 0.1,
    )

#### Add datasets and sensors

In [67]:
data = dataset['u'] # shape (500, 201, 100)
# mu = dataset['mu'] # shape (500, 201, 2)

manager.add_data(
    data=data,
    # random=3,
    stationary=[(15,),(30,),(45,)],
    # params=mu,
    id = 'KS',
    compress = False
)

#### Analyze sensor summary

In [68]:
manager.sensor_measurements_df

,DS-0,DS-1,DS-2
0,1.258974,-0.169609,-0.222487
1,1.310099,-0.145623,-0.422734
2,1.377290,-0.085841,-0.576607
3,1.453632,-0.038841,-0.728031
4,1.522495,-0.006997,-0.901799
...,...,...,...
100495,0.890233,-0.398904,2.244036
100496,0.825840,-0.105987,2.078920
100497,0.100336,0.115727,1.665149
100498,-1.266814,0.278429,1.136558


In [58]:
manager.sensor_summary_df

,data id,sensor_number,type,loc/traj
0,DS,0,stationary (random),"(87,)"
1,DS,1,stationary (random),"(63,)"
2,DS,2,stationary (random),"(21,)"
3,DS,3,stationary,"(15,)"
4,DS,4,stationary,"(30,)"
5,DS,5,stationary,"(45,)"


#### Get train, validation, and test set

In [ ]:
train_dataset, val_dataset, test_dataset= manager.prepare()

#### Initialize SHRED

When using a `ParametricDataManager`, ensure `latent_forecaster` is set to None.

In [50]:
shred = SHRED(sequence_model="LSTM", decoder_model="MLP", latent_forecaster=None)

#### Fit SHRED

In [ ]:
val_errors = shred.fit(train_dataset=train_dataset, val_dataset=val_dataset, num_epochs=10, sindy_regularization=0)
print('val_errors:', val_errors)

Fitting SHRED...
Epoch 1: Average training loss = 0.026830
Validation MSE (epoch 1): 0.017097
Epoch 2: Average training loss = 0.011875
Validation MSE (epoch 2): 0.014346
Epoch 3: Average training loss = 0.009531
Validation MSE (epoch 3): 0.010721
Epoch 4: Average training loss = 0.007884
Validation MSE (epoch 4): 0.009414
Epoch 5: Average training loss = 0.006994
Validation MSE (epoch 5): 0.008351
Epoch 6: Average training loss = 0.006297
Validation MSE (epoch 6): 0.006967
Epoch 7: Average training loss = 0.005512
Validation MSE (epoch 7): 0.006366
Epoch 8: Average training loss = 0.004923
Validation MSE (epoch 8): 0.005887
Epoch 9: Average training loss = 0.004537
Validation MSE (epoch 9): 0.005361
Epoch 10: Average training loss = 0.004175
Validation MSE (epoch 10): 0.005111
val_errors: [0.01709735 0.01434578 0.0107211  0.00941385 0.00835117 0.00696706
 0.00636586 0.00588656 0.0053609  0.00511133]


#### Evaluate SHRED

In [52]:
train_mse = shred.evaluate(dataset=train_dataset)
val_mse = shred.evaluate(dataset=val_dataset)
test_mse = shred.evaluate(dataset=test_dataset)
print(f"Train MSE: {train_mse:.3f}")
print(f"Val   MSE: {val_mse:.3f}")
print(f"Test  MSE: {test_mse:.3f}")

Train MSE: 0.003
Val   MSE: 0.005
Test  MSE: 0.003


#### Initialize Parametric SHRED Engine for Downstream Tasks (TODO)

In [ ]:
engine = ParametricSHREDEngine(manager, shred)